In [ ]:
#Instalations
!pip install transformers
!pip install datasets
!pip install --upgrade pandas
!pip install evaluate
!pip install torch
!pip install "accelerate>=0.26.0"
!pip install transformers[torch]
!pip install seaborn
!pip install matplotlib


In [ ]:
#Installations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler

In [ ]:
#Configs
data_path = "../../0. Dataset/cefr_leveled_texts200.csv"
text_column_name = "text"
label_column_name = "level"

model_name = "distilbert-base-uncased"
num_labels = 6  # A1 A2 B1 B2 C1 C2

In [ ]:
#Read and Prepare Dataset
import pandas as pd
df = pd.read_csv(data_path)
df = df.rename(columns={"label": "level"})

In [ ]:
df.head()

,text,level,level
0,¡Hola!\nHe estado queriendo escribir durante a...,B2,3
1,No fue tanto lo difícil que la gente encontró ...,B2,3
2,Keith recientemente regresó de un viaje a Chic...,B2,3
3,El Observatorio Griffith es un planetario y un...,B2,3
4,-LRB- The Hollywood Reporter -RRB- Es oficial:...,B2,3


In [ ]:
#Clean Dataset
from bs4 import BeautifulSoup
class Cleaner():
  def __init__(self):
    pass
  def put_line_breaks(self,text):
    text = text.replace('</p>','</p>\n')
    return text
  def remove_html_tags(self,text):
    cleantext = BeautifulSoup(text, "lxml").text
    return cleantext
  def clean(self,text):
    text = self.put_line_breaks(text)
    text = self.remove_html_tags(text)
    return text

cleaner = Cleaner()
df['text_cleaned'] = df[text_column_name].apply(cleaner.clean)

In [ ]:
#Label Encoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df[label_column_name].tolist())
df['label'] = le.transform(df[label_column_name].tolist())

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [ ]:
df.head()

,text,level,level,text_cleaned
0,¡Hola!\nHe estado queriendo escribir durante a...,B2,3,¡Hola!\nHe estado queriendo escribir durante a...
1,No fue tanto lo difícil que la gente encontró ...,B2,3,No fue tanto lo difícil que la gente encontró ...
2,Keith recientemente regresó de un viaje a Chic...,B2,3,Keith recientemente regresó de un viaje a Chic...
3,El Observatorio Griffith es un planetario y un...,B2,3,El Observatorio Griffith es un planetario y un...
4,-LRB- The Hollywood Reporter -RRB- Es oficial:...,B2,3,-LRB- The Hollywood Reporter -RRB- Es oficial:...


In [ ]:
#Train/Test Split
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,test_size=0.4)

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(16, activation='relu'), # if x <= 0 --> 0, x > 0 --> x
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])